In [33]:
import SimpleITK as sitk
import pydicom
import numpy as np
import glob, os
from skimage.filters import gaussian
from tqdm import tqdm
import SimpleITK as sitk
from einops.einops import rearrange

In [34]:
sd = 2
files = glob.glob(rf'data/DigitalPhantomCT_SD_{sd}_US_05/*')
print(files)
vof_location = (410, 247, 16)  # start, start, size
aif_location = (123, 251, 8)  # start, start, size
aif_values_rescaled_nosmoothing_undersampled = [40.863964,-1024, 39.63787,0, 39.831463,0, 41.154358,0, 496.7779,0, 293.02020,0, 101.23303,0, 49.86609,0, 42.47725,0, 39.476543,0, 40.928497,0, 40.121857,0, 41.38022,0, 39.18615,0, 41.638344,0]
for ix, file in tqdm(enumerate(files[::16])):
    ds = pydicom.dcmread(file)
    data = ds.pixel_array
    new_aif_value = aif_values_rescaled_nosmoothing_undersampled[ix]
    new_aif_array = np.tile(np.array(new_aif_value), (8, 8))
    data[aif_location[0]:aif_location[0] + aif_location[2],
               aif_location[1]:aif_location[1] + aif_location[2]] = new_aif_array
    # copy the data back to the original data set
    ds.PixelData = data.tobytes()
    # print('The information of the data set after downsampling: \n')
    ds.save_as(file)

['data/DigitalPhantomCT_SD_2_US_05\\CT000001', 'data/DigitalPhantomCT_SD_2_US_05\\CT000002', 'data/DigitalPhantomCT_SD_2_US_05\\CT000003', 'data/DigitalPhantomCT_SD_2_US_05\\CT000004', 'data/DigitalPhantomCT_SD_2_US_05\\CT000005', 'data/DigitalPhantomCT_SD_2_US_05\\CT000006', 'data/DigitalPhantomCT_SD_2_US_05\\CT000007', 'data/DigitalPhantomCT_SD_2_US_05\\CT000008', 'data/DigitalPhantomCT_SD_2_US_05\\CT000009', 'data/DigitalPhantomCT_SD_2_US_05\\CT000010', 'data/DigitalPhantomCT_SD_2_US_05\\CT000011', 'data/DigitalPhantomCT_SD_2_US_05\\CT000012', 'data/DigitalPhantomCT_SD_2_US_05\\CT000013', 'data/DigitalPhantomCT_SD_2_US_05\\CT000014', 'data/DigitalPhantomCT_SD_2_US_05\\CT000015', 'data/DigitalPhantomCT_SD_2_US_05\\CT000016', 'data/DigitalPhantomCT_SD_2_US_05\\CT000017', 'data/DigitalPhantomCT_SD_2_US_05\\CT000018', 'data/DigitalPhantomCT_SD_2_US_05\\CT000019', 'data/DigitalPhantomCT_SD_2_US_05\\CT000020', 'data/DigitalPhantomCT_SD_2_US_05\\CT000021', 'data/DigitalPhantomCT_SD_2_US_05

30it [00:00, 201.35it/s]


In [35]:

simulated_data_size = 32 * 7
scan_center = 512 // 2
simulated_data_start = scan_center - simulated_data_size // 2
simulated_data_end = scan_center + simulated_data_size // 2
other_files = sorted([file for file in files if file not in files[::16]])

for ix, file in tqdm(enumerate(other_files)):
    ds = pydicom.dcmread(file)
    data = ds.pixel_array
    phantomcurves = data[simulated_data_start:simulated_data_end,
                         simulated_data_start:simulated_data_end]
    truncate = np.ceil(2 * sd) / sd if sd != 0 else 0
    phantomcurves = gaussian(phantomcurves, sigma=(sd, sd), mode='nearest', truncate=truncate, preserve_range=True).astype(np.int16)
    data[simulated_data_start:simulated_data_end, simulated_data_start:simulated_data_end] = phantomcurves
    ds.PixelData = data.tobytes()
    ds.save_as(file)


450it [00:02, 168.16it/s]


In [36]:
files = glob.glob(rf'data/DigitalPhantomCT_SD_{sd}_US_05/*')
splits = np.array(np.array_split(files, 30))
indices = np.arange(0,30,2)
new_splits = splits[indices].flatten()
print(new_splits)

['data/DigitalPhantomCT_SD_2_US_05\\CT000001'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000002'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000003'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000004'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000005'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000006'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000007'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000008'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000009'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000010'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000011'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000012'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000013'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000014'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000015'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000016'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000033'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000034'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000035'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000036'
 'data/DigitalPhantomCT_SD_2_US_05\\CT000037'
 'data/DigitalPhantomCT_SD_2_US_05

In [37]:
for file in files:
    if file not in new_splits:
        os.remove(file)
